In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPool2D
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json
import matplotlib.pyplot as plt


In [2]:
! pip install keras_preprocessing
! pip install scikit-learn
! pip install matplotlib

In [3]:
train_dir = 'images/train'
test_dir = 'images/test'

In [4]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels

In [5]:
train = pd.DataFrame()
train['image'],train['label'] = createdataframe(train_dir)
print(train)
test = pd.DataFrame()
test['image'],test['label'] = createdataframe(test_dir)
print(test)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                                             image     label
0         images/train\angry\Training_10118481.jpg     angry
1         images/train\angry\Training_10120469.jpg     angry
2         images/train\angry\Training_10131352.jpg     angry
3         images/train\angry\Training_10161559.jpg     angry
4          images/train\angry\Training_1021836.jpg     angry
...                                            ...       ...
28704  images/train\surprise\Training_99916297.jpg  surprise
28705  images/train\surprise\Training_99924420.jpg  surprise
28706  images/train\surprise\Training_99937001.jpg  surprise
28707  images/train\surprise\Training_99951755.jpg  surprise
28708  images/train\surprise\Training_99984132.jpg  surprise

[28709 rows x 2 columns]
angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
          

In [6]:
import numpy as np
from tqdm import tqdm
from PIL import Image
from keras.preprocessing.image import img_to_array

def extract_features(images):
    features = []
    for image_path in tqdm(images):
        img = Image.open(image_path).convert('L')  # Open image in grayscale mode
        img_array = img_to_array(img)  # Convert image to numpy array
        features.append(img_array)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features


In [7]:

# Assuming train['image'] is a pandas Series object containing file paths
image_paths = train['image'].tolist()  # Convert Series to list of paths
train_features = extract_features(image_paths)


100%|██████████| 28709/28709 [09:33<00:00, 50.07it/s]


In [8]:
test_features = extract_features(test['image'])

100%|██████████| 7178/7178 [02:02<00:00, 58.53it/s]


In [9]:
x_train = train_features/255.0
x_test = test_features/255.0


In [10]:
lr = LabelEncoder()
lr.fit(train['label'])


LabelEncoder()

In [11]:
y_train = lr.transform(train['label'])
y_test = lr.transform(test['label'])

In [12]:
y_train = to_categorical(y_train,num_classes=7)
y_test = to_categorical(y_test,num_classes=7)


In [13]:
model = Sequential()
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(7,activation='softmax'))

c:\Users\91843\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
model.fit(x= x_train,y = y_train,batch_size=128,epochs=20,validation_data=(x_test,y_test))

Epoch 1/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 522s 2s/step - accuracy: 0.2397 - loss: 1.8261 - val_accuracy: 0.2471 - val_loss: 1.8092
Epoch 2/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 482s 2s/step - accuracy: 0.2529 - loss: 1.7944 - val_accuracy: 0.2986 - val_loss: 1.7017
Epoch 3/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 431s 2s/step - accuracy: 0.3062 - loss: 1.7076 - val_accuracy: 0.3640 - val_loss: 1.5975
Epoch 4/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 404s 2s/step - accuracy: 0.3563 - loss: 1.6145 - val_accuracy: 0.4315 - val_loss: 1.4542
Epoch 5/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 421s 2s/step - accuracy: 0.4125 - loss: 1.5136 - val_accuracy: 0.4579 - val_loss: 1.3803
Epoch 6/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 640s 3s/step - accuracy: 0.4368 - loss: 1.4512 - val_accuracy: 0.4894 - val_loss: 1.3211
Epoch 7/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 842s 4s/step - accuracy: 0.4510 - loss: 1.4228 - val_accuracy: 0.5022 - val_loss: 1.3010
Epoch 8/20
147/225 ━━━━━━━━━━━━━━━━━━━━ 5:35 4s/step - accuracy: 0.4613 - loss: 1.3973

In [51]:
model_json = model.to_json()
with open("facialemotionmodel.json",'w') as json_file:
    json_file.write(model_json)
model.save("facialemotionmodel.h5")

In [52]:
json_file = open("facialemotionmodel.json",'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")

In [53]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [61]:
! pip install pillow

In [69]:
! pip install tensorflow
! pip install keras

In [70]:
import numpy as np
from PIL import Image
from keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# Assuming 'model' is defined elsewhere in your code
model = load_model('facialemotionmodel.h5')

# Define your actual labels
label = ['class1', 'class2', 'class3']  # Replace with your actual labels

def ef(image_path):
    img = Image.open(image_path).convert('L')  # Load image in grayscale mode using PIL
    img_array = img_to_array(img) / 255.0
    img_array = img_array.reshape(1, 48, 48, 1)
    return img_array

image_path = 'images/train/sad/Training_1362985.jpg'
print("Original image is of sad")
img = ef(image_path)
pred = model.predict(img)
pred_label = label[np.argmax(pred)]
print("Model prediction is", pred_label)


Original image is of sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 792ms/step


IndexError: list index out of range

In [71]:
# image = 'images/train/sad/42.jpg'
# print("original image is of sad")
# img = ef(image)
# pred = model.predict(img)
# pred_label = label[pred.argmax()]
# print("model prediction is ",pred_label)

In [72]:
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
image = 'images/train/sad/Training_26728.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

original image is of sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step


IndexError: list index out of range

In [76]:
image = 'images/train/fear/Training_176862.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

original image is of fear
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


IndexError: list index out of range

In [ ]:
image = 'images/train/disgust/299.jpg'
print("original image is of disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/happy/7.jpg'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/surprise/15.jpg'
print("original image is of surprise")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')